<a href="https://colab.research.google.com/github/MShokry/GameOfDeepLearning/blob/master/Gameof_DL_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#! pip install keras

# Importing The Libs 

In [0]:
import keras
from keras.layers import Flatten, Dropout, Dense, Activation
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Mounting GDrive For reading the Images

In [0]:
from google.colab import drive
drive.mount('/content/gdriv/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdriv/


## UnZip The Images Folder

In [0]:
! unzip /content/gdriv/My\ Drive/mlms/train.zip
! ls

Archive:  /content/gdriv/My Drive/mlms/train.zip
   creating: images/
  inflating: images/2870091.jpg      
   creating: __MACOSX/
   creating: __MACOSX/images/
  inflating: __MACOSX/images/._2870091.jpg  
  inflating: images/2860282.jpg      
  inflating: __MACOSX/images/._2860282.jpg  
  inflating: images/2894465.jpg      
  inflating: __MACOSX/images/._2894465.jpg  
  inflating: images/2784253.jpg      
  inflating: __MACOSX/images/._2784253.jpg  
  inflating: images/2853890.jpg      
  inflating: __MACOSX/images/._2853890.jpg  
  inflating: images/2898539.jpg      
  inflating: __MACOSX/images/._2898539.jpg  
  inflating: images/644577.jpg       
  inflating: __MACOSX/images/._644577.jpg  
  inflating: images/2856056.jpg      
  inflating: __MACOSX/images/._2856056.jpg  
  inflating: images/2792357.jpg      
  inflating: __MACOSX/images/._2792357.jpg  
  inflating: images/2904710.jpg      
  inflating: __MACOSX/images/._2904710.jpg  
  inflating: images/2748659.jpg      
  inflatin

# Loading The Training, Test And Validation Data

In [0]:
from google.colab import files
#Import the data
train = pd.read_csv('train.csv')
train = train.sample(frac=1)
TSsplit = int(np.ceil(6252*.85))
#Converting the Pandas frame to numpy 
image = np.array(train.iloc[:TSsplit,0],np.str)
label = np.array(train.iloc[:TSsplit,1:],np.float32)
#Test Data
image_test = np.array(train.iloc[TSsplit:,0],np.str)
label_test = np.array(train.iloc[TSsplit:,1:],np.float32)


## Loading Images to Numpy Array

In [0]:
#Load Images
images = []
x=0
for i in image:
    im = cv2.imread('./images/'+i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (210, 140), interpolation = cv2.INTER_AREA)
    images.append(im)
    
images = np.array(images)

images_test = []
for i in image_test:
    im = cv2.imread('./images/'+i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (210, 140), interpolation = cv2.INTER_AREA)
    images_test.append(im)
images_test = np.array(images_test)

## One Hot Encoding For the TRain/Test data

In [0]:

#Train
from keras.utils import np_utils
print("Before conding")
print(label[:10])
labels = np_utils.to_categorical(label,6)
print("Encoded Data")
print(labels[:10])


Before conding
[[2.]
 [2.]
 [2.]
 [5.]
 [4.]
 [3.]
 [2.]
 [2.]
 [5.]
 [2.]]
Encoded Data
[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]]


# Create Model

#### Model 1 :
```python
#Creat Model
inception = keras.applications.InceptionV3(input_shape=(140, 210, 3), include_top=False, pooling='avg',)

model = keras.Sequential()
model.add(inception)
model.add(Dense(512, name='aux_output'))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(6, name='aux_output2'))
model.add(Activation('softmax'))
#Model Summary
model.summary()
```
**Results**  : .175

#### Model 2: 

```Python
... Inception
model.add(Dropout(0.2))
model.add(Dense(512, name='aux_output'))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(128, name='aux_output1'))
model.add(Activation('relu'))
model.add(Dropout(0.3))
... d6
```
**Results**  : 0.10

#### Model 3: 

```Python
... inception
model.add(Dropout(0.15))
model.add(Dense(1024, name='aux_output'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512, name='aux_output'))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(128, name='aux_output1'))
model.add(Activation('relu'))
model.add(Dropout(0.4))
.... D6
```
**Results** 

### Model 4:

```Python
#inception = keras.applications.InceptionV3(input_shape=(140, 210, 3), include_top=False, pooling='max',) #avg
resnet = keras.applications.ResNet50(input_shape=(140, 210, 3), include_top=False, pooling='max')

model = keras.Sequential()
model.add(resnet)
model.add(Dropout(0.15))
model.add(Dense(1024, name='aux_output'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512, name='aux_output1'))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(128, name='aux_output2'))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(6, name='aux_output3'))
model.add(Activation('softmax'))
#Model Summary
model.summary()
```
**Results** :.88 

In [0]:
# Creat Model 3
#inception = keras.applications.InceptionV3(input_shape=(140, 210, 3), include_top=False, pooling='max',)

#model = keras.Sequential()
#model.add(inception)
#model.add(Dropout(0.15))
#model.add(Dense(1024, name='aux_output'))
#model.add(Activation('relu'))
#model.add(Dropout(0.2))
#model.add(Dense(512, name='aux_output1'))
#model.add(Activation('relu'))
#model.add(Dropout(0.3))
#model.add(Dense(128, name='aux_output2'))
#model.add(Activation('relu'))
#model.add(Dropout(0.4))
#model.add(Dense(6, name='aux_output3'))
#model.add(Activation('softmax'))
# Model Summary
#model.summary()

In [0]:
#Creat Model 4
#inception = keras.applications.InceptionV3(input_shape=(140, 210, 3), include_top=False, pooling='max',) #avg
resnet = keras.applications.ResNet50(input_shape=(140, 210, 3), include_top=False, pooling='max')

model = keras.Sequential()
model.add(resnet)
model.add(Dropout(0.2))
#model.add(Dense(1024, name='aux_output'))
#model.add(Activation('relu'))
#model.add(Dropout(0.3))
#model.add(Dense(512, name='aux_output1'))
#model.add(Activation('relu'))
#model.add(Dropout(0.2)) #.3 @model 4
#model.add(Dense(128, name='aux_output2'))
#model.add(Activation('relu'))
#model.add(Dropout(0.4))
#model.add(Dense(6, name='aux_output3'))
#model.add(Activation('softmax'))
#Model Summary
#model.summary()

model.add(Dense(2048, name='inp'))
model.add(Activation('sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(512, name='Dense1'))
model.add(Activation('relu'))
model.add(Dropout(0.2)) #.3 @model 4
model.add(Dense(128, name='Dense2'))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(64, name='Dense3'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6, name='out'))
model.add(Activation('softmax'))
#Model Summary
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
94658560/94653016 [==============================] - 9s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
inp (Dense)                  (None, 2048)              4196352   
_________________________________________________________________
activation_50 (Activation)   (None, 2048)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
______________________

## Moel Compile 

In [0]:
#TRaining
#eras_images = keras.applications.inception_v3.preprocess_input(images)
#mages_test = keras.applications.inception_v3.preprocess_input(images_test)

from keras.optimizers import RMSprop, Adam,SGD
def model_fit(model,batch_size=64,epochs=10):
    #optimizer = RMSprop(lr=0.0002, rho=0.9, epsilon=1e-08, decay=0.0) #0.001 DEC .002
    #optimizer = RMSprop()
    #optimizer = Adam(lr=0.0001)
    optimizer = SGD(lr = 0.008, momentum = 0.04 , decay=0.0001 )
    model.compile(loss="categorical_crossentropy",optimizer=optimizer,metrics=['accuracy'])
    from keras.callbacks import ModelCheckpoint
    checkpointer = ModelCheckpoint(filepath='model.best', verbose=1, monitor='val_acc', save_best_only=True)
    training = model.fit(images, labels,batch_size=batch_size, epochs=epochs,validation_split=0.25, callbacks=[checkpointer],verbose=1, shuffle=True)
    return training

In [0]:
#optimizer = RMSprop(lr=0.0002, rho=0.9, epsilon=1e-08, decay=0.0)
#optimizer = Adam(lr=0.0001)
#model.compile(loss="categorical_crossentropy",optimizer=optimizer,metrics=['accuracy'])

## Starting Magic Here The training starts here

In [0]:
training = model_fit(model,batch_size=64,epochs=100)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 3986 samples, validate on 1329 samples
Epoch 1/100
3986/3986 [==============================] - 48s 12ms/step - loss: 1.7757 - acc: 0.2521 - val_loss: 1.5780 - val_acc: 0.3303

Epoch 00001: val_acc improved from -inf to 0.33032, saving model to model.best
Epoch 2/100
3986/3986 [==============================] - 34s 8ms/step - loss: 1.5855 - acc: 0.3329 - val_loss: 1.4055 - val_acc: 0.4755

Epoch 00002: val_acc improved from 0.33032 to 0.47555, saving model to model.best
Epoch 3/100
3986/3986 [==============================] - 34s 9ms/step - loss: 1.4171 - acc: 0.4112 - val_loss: 1.1607 - val_acc: 0.4951

Epoch 00003: val_acc improved from 0.47555 to 0.49511, saving model to model.best
Epoch 4/100
3986/3986 [==============================] - 34s 9ms/step - loss: 1.1372 - acc: 0.5484 - val_loss: 0.7653 - val_acc: 0.7126

Epoch 00004: val_acc improved from

# Model Evaluation 

In [0]:
model.load_weights('model.best')
label_predict_sma = model.predict(images_test[:10])
label_predict = model.predict_classes(images_test)

In [0]:
lbl = np_utils.to_categorical(label_predict,6)
model.evaluate(images_test,lbl)

937/937 [==============================] - 3s 3ms/step


[0.009047855193357842, 1.0]

In [0]:
from sklearn.metrics import classification_report,confusion_matrix,log_loss
#print(log_loss(label_predict,label_test))
conv = pd.DataFrame(confusion_matrix(label_test,label_predict))
print(conv)
print(classification_report(label_test,label_predict))


     0    1    2    3    4
0  269    1    2    3   26
1    4  154    0    3    1
2    5    0  144    0    0
3    3    2    0  123    5
4   32    0    0    1  159
              precision    recall  f1-score   support

         1.0       0.86      0.89      0.88       301
         2.0       0.98      0.95      0.97       162
         3.0       0.99      0.97      0.98       149
         4.0       0.95      0.92      0.94       133
         5.0       0.83      0.83      0.83       192

    accuracy                           0.91       937
   macro avg       0.92      0.91      0.92       937
weighted avg       0.91      0.91      0.91       937



# Model Save

In [0]:
model.save('model.hp5')
model.save_weights('model_weights.hp5')
! cp model* /content/gdriv/My\ Drive/mlms/

# Generating Submission

In [0]:
#Loading The Data
evaluate = pd.read_csv('/content/gdriv/My Drive/mlms/test_ApKoW4T.csv')
image_eval = np.array(evaluate.iloc[:,0],np.str)
images_eval = []
for i in image_eval:
    im = cv2.imread('./images/'+i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (210, 140), interpolation = cv2.INTER_AREA)
    images_eval.append(im)
images_eval = np.array(images_eval)

In [0]:
#Predict Labels
eval_label_predict = model.predict_classes(images_eval)

In [0]:
eval_label_predict.shape,image_eval.shape

((2680,), (2680,))

In [0]:
np.savetxt("submission.csv", np.dstack((image_eval,eval_label_predict))[0],"%s,%s",header="image,category",comments="")

In [0]:
!cp subm*  /content/gdriv/My\ Drive/mlms/

In [0]:
#Drawing Test Dta 
fig = plt.figure(figsize=(20,20))
for i in range(10):
    rn = np.random.randint(1,100)
    ax = fig.add_subplot(1,10,i+1)
    ax.imshow(np.reshape(data_test[rn],(28,28)),cmap='gray')
    ax.set_title(label_test[rn])


In [0]:
model.load_weights('model.best')